In [78]:
class Car:
    # id : str
    # pos: int
    # status: 'Car' --> 0 empty
    #               --> 1 full
    def __init__(self, id, pos):
        self.id = id
        self.pos = pos

    def __str__(self):
        return 'Car:{id!s} {pos!s} {status!s}'.format(self)

In [79]:
class Order:

    def __init__(self, id, bgn, end):
        self.id = id
        self.bgn = bgn
        self.end = end
    

In [80]:
from pickle import load
input = open('data/pos_data', 'rb')
pos_list = load(input)
input.close()

In [81]:
import networkx as nx
H = nx.read_graphml('data/Chengdu_graph', node_type=int)

In [82]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371
    return c * r * 1000
def dist(a, b):
    dicta = H.nodes[a]
    dictb = H.nodes[b]
    return haversine(float(dicta['x']), float(dicta['y']), float(dictb['x']), float(dictb['y']))
def digra(x, y):
    return nx.astar_path_length(H, x, y, heuristic = dist, weight = 'length')

In [83]:
def min_distance(car, order1, order2):
    return digra(car.pos, order1.bgn) + digra(order1.bgn, order2.bgn) + digra(order1.end, order2.end) + min(digra(order1.end, order2.bgn), digra(order2.end, order2.bgn))

In [89]:
from queue import PriorityQueue as PQ
import csv

In [90]:
cars = PQ()
with open('data/car.csv') as f:
    f_csv = csv.DictReader(f)
    for row in f_csv:
        cars.put((int(row['time']), Car(row['id'], int(row['pos']))))

TypeError: '<' not supported between instances of 'Car' and 'Car'

In [92]:
orders = PQ()
with open('data/order.csv') as f:
    f_csv = csv.DictReader(f)
    for row in f_csv:
        orders.put((int(row['time']), Order(row['id'], int(row['bgn']), int(row['end']))))

In [0]:
vel = 6.3
total_of_orders = 0
total_of_cars = 0
total_pass_waiting_time = 0
time_now = 0

while (not orders.empty()) and (not cars.empty()):
    p_car = cars.get()
    p_order1 = orders.get()
    p_order2 = orders.get()
    time_now = max(p_car[0], p_order1[0], p_order2[0])
    total_pass_waiting_time += 2 * time_now - p_order1[0] - p_order2[0]

    car = p_car[1]
    order1 = p_order1[1]
    order2 = p_order2[1]

    d = min(min_distance(car, order1, order2), min_distance(car, order2, order1))
    total_of_cars += d
    total_of_orders += digra(order1.bgn, order1.end) + digra(order2.bgn, order2.end)

    cars.put((p_car[0] + int(d/vel), car))

In [0]:
print(total_of_cars, total_of_orders, total_pass_waiting_time)